To use this, you need to download the desired flownet pre-trained model weights(flownet2 for this project) from https://github.com/NVIDIA/flownet2-pytorch and keep it in a google drive folder. 

You can also download it directly in colab, or on your system, but the flownet model with pretrained weights are transferred by google drive file share which might make this harder.

In [1]:
#pull github code
!git clone https://github.com/NVIDIA/flownet2-pytorch.git

Cloning into 'flownet2-pytorch'...
remote: Enumerating objects: 557, done.
remote: Total 557 (delta 0), reused 0 (delta 0), pack-reused 557
Receiving objects: 100% (557/557), 6.28 MiB | 10.70 MiB/s, done.
Resolving deltas: 100% (312/312), done.


After cloning the repo, for colab, you need to perform these changes 

In setup.py in /content/flownet2-pytorch/networks/(channelnorm_package, correlation_package, and resample_2d_package) 
1. change cxx_args = ['-std=c++11'] to cxx_args = ['-std=c++14'] 
2. Add '-gencode', 'arch=compute_37,code=sm_37' to nvcc-args 



In [2]:
!nvidia-smi

Sun Feb 14 10:08:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#install tensorboardX, setproctitle, colorama
!pip install tensorboardX
!pip install setproctitle
!pip install colorama

     |████████████████████████████████| 317kB 7.5MB/s 


In [4]:
!pip install scipy==1.1.0
!pip install Pillow

     |████████████████████████████████| 31.2MB 110kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [5]:
%cd ./flownet2-pytorch/

/content/flownet2-pytorch


Then install

In [6]:
!bash install.sh

running install
running bdist_egg
running egg_info
creating correlation_cuda.egg-info
writing correlation_cuda.egg-info/PKG-INFO
writing dependency_links to correlation_cuda.egg-info/dependency_links.txt
writing top-level names to correlation_cuda.egg-info/top_level.txt
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
/usr/local/lib/python3.6/dist-packages/torch/utils/cpp_extension.py:339: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'correlation_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SO

In [7]:
!ls

convert.py		  __init__.py	    main.py	   run-caffe2pytorch.sh
datasets.py		  install.sh	    models.py	   utils
Dockerfile		  launch_docker.sh  networks
download_caffe_models.sh  LICENSE	    README.md
image.png		  losses.py	    run_a_pair.py


In [8]:
!python main.py --help

usage: main.py [-h] [--start_epoch START_EPOCH] [--total_epochs TOTAL_EPOCHS]
               [--batch_size BATCH_SIZE] [--train_n_batches TRAIN_N_BATCHES]
               [--crop_size CROP_SIZE [CROP_SIZE ...]]
               [--gradient_clip GRADIENT_CLIP]
               [--schedule_lr_frequency SCHEDULE_LR_FREQUENCY]
               [--schedule_lr_fraction SCHEDULE_LR_FRACTION]
               [--rgb_max RGB_MAX] [--number_workers NUMBER_WORKERS]
               [--number_gpus NUMBER_GPUS] [--no_cuda] [--seed SEED]
               [--name NAME] [--save SAVE]
               [--validation_frequency VALIDATION_FREQUENCY]
               [--validation_n_batches VALIDATION_N_BATCHES]
               [--render_validation] [--inference] [--inference_visualize]
               [--inference_size INFERENCE_SIZE [INFERENCE_SIZE ...]]
               [--inference_batch_size INFERENCE_BATCH_SIZE]
               [--inference_n_batches INFERENCE_N_BATCHES] [--save_flow]
               [--resume PATH] [--log

The pretrained weights are kept in a pretrained directory and the results are kept in a result directory

In [9]:
!mkdir pretrained
!mkdir results

Also I had the extracted consecutive frames of the fetoscopic dataset (videoXX) gotten from https://weiss-develop.cs.ucl.ac.uk/fetoscopy-data/fetoscopy-placenta-dataset/fetoscopy-placenta-dataset.zip and kept them on my google drive. 

Virtually all my data was on google drive hahaha, thank you google. 

Adding fetoscope data

In [10]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


listed = drive.ListFile({'q': "title contains 'anon001.zip'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title anon001.zip, id 1flMv1QbSrxAH5IzV4xn5g9mljrZK-0rz


In [11]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = file['id']

fname = os.path.join('/content/flownet2-pytorch', file['title'])
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)

downloading to /content/flownet2-pytorch/anon001.zip


In [12]:
# unzip sample images
!unzip anon001.zip

Archive:  anon001.zip
   creating: anon001/
  inflating: anon001/anon001_00831.png  
  inflating: anon001/anon001_00832.png  
  inflating: anon001/anon001_00833.png  
  inflating: anon001/anon001_00834.png  
  inflating: anon001/anon001_00835.png  
  inflating: anon001/anon001_00836.png  
  inflating: anon001/anon001_00837.png  
  inflating: anon001/anon001_00838.png  
  inflating: anon001/anon001_00839.png  
  inflating: anon001/anon001_00840.png  
  inflating: anon001/anon001_00841.png  
  inflating: anon001/anon001_00842.png  
  inflating: anon001/anon001_00843.png  
  inflating: anon001/anon001_00844.png  
  inflating: anon001/anon001_00845.png  
  inflating: anon001/anon001_00846.png  
  inflating: anon001/anon001_00847.png  
  inflating: anon001/anon001_00848.png  
  inflating: anon001/anon001_00849.png  
  inflating: anon001/anon001_00850.png  
  inflating: anon001/anon001_00851.png  
  inflating: anon001/anon001_00852.png  
  inflating: anon001/anon001_00853.png  
  inflating: 

**resize images to 448x448**

In [13]:
from PIL import Image
import numpy as np

In [16]:
IMG_SHAPE_STANDARD = (448,448);
frames_path = "/content/flownet2-pytorch/anon001"

In [17]:
def resize_all_images(path):
  dirs = os.listdir(path)
  for item in dirs:
      if item.lower().endswith(('.png', '.jpg', '.jpeg')):
        im = Image.open(os.path.join(path,item))
        # f, e = os.path.splitext(path+item)
        imResize = im.resize(IMG_SHAPE_STANDARD, Image.ANTIALIAS)
        imResize.save(os.path.join(path,item), 'PNG')

resize_all_images(frames_path)

Note: the pretrained flownet models can be gotten from the github account. Just clicking on them links them to your gdrive, or one can save it and use it later

In [18]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains 'FlowNet2_checkpoint.pth.tar'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title FlowNet2_checkpoint.pth.tar, id 1hF8vS6YeHkx3j2pfCeQqqZGwA_PJq_Da


In [19]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1hF8vS6YeHkx3j2pfCeQqqZGwA_PJq_Da' 

fname = os.path.join('/content/flownet2-pytorch/pretrained', 'FlowNet2_checkpoint.pth.tar')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)



downloading to /content/flownet2-pytorch/pretrained/FlowNet2_checkpoint.pth.tar


Perform evaluation with the pretrained models

In [20]:
## Run our own data
!python main.py --inference --model FlowNet2 --save_flow \
--inference_dataset ImagesFromFolder \
--inference_dataset_root /content/flownet2-pytorch/anon001 \
--resume /content/flownet2-pytorch/pretrained/FlowNet2_checkpoint.pth.tar \
--save /content/flownet2-pytorch/results

Parsing Arguments
  [0.030s] batch_size: 8
  [0.030s] crop_size: [256, 256]
  [0.030s] fp16: False
  [0.030s] fp16_scale: 1024.0
  [0.030s] gradient_clip: None
  [0.030s] inference: True
  [0.030s] inference_batch_size: 1
  [0.030s] inference_dataset: ImagesFromFolder
  [0.030s] inference_dataset_iext: png
  [0.030s] inference_dataset_replicates: 1
  [0.030s] inference_dataset_root: /content/flownet2-pytorch/anon001
  [0.030s] inference_n_batches: -1
  [0.030s] inference_size: [-1, -1]
  [0.030s] inference_visualize: False
  [0.030s] log_frequency: 1
  [0.030s] loss: L1Loss
  [0.030s] model: FlowNet2
  [0.030s] model_batchNorm: False
  [0.030s] model_div_flow: 20.0
  [0.030s] name: run
  [0.030s] no_cuda: False
  [0.030s] number_gpus: 1
  [0.030s] number_workers: 8
  [0.030s] optimizer: Adam
  [0.031s] optimizer_amsgrad: False
  [0.031s] optimizer_betas: (0.9, 0.999)
  [0.031s] optimizer_eps: 1e-08
  [0.031s] optimizer_lr: 0.001
  [0.031s] optimizer_weight_decay: 0
  [0.031s] render_va

In [ ]:
# import shutil
# shutil.rmtree('/content/flownet2-pytorch/networks/resample2d_package/resample2d_cuda.egg-info')

In [ ]:
### How to get the .flo files to work.

In [21]:
!zip -r /content/flow_files_anon001.zip /content/flownet2-pytorch/results/inference/run.epoch-0-flow-field

  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/ (stored 0%)
  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/000276.flo (deflated 8%)
  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/000310.flo (deflated 9%)
  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/000024.flo (deflated 11%)
  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/000363.flo (deflated 9%)
  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/000021.flo (deflated 11%)
  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/000413.flo (deflated 10%)
  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/000376.flo (deflated 11%)
  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/000254.flo (deflated 10%)
  adding: content/flownet2-pytorch/results/inference/run.epoch-0-flow-field/000313.flo (deflated 9

In [ ]:
# import shutil

# shutil.rmtree('/content/flownet2-pytorch/anon010')
# shutil.rmtree('/content/flownet2-pytorch/results')

In [24]:
2+2

4